<a href="https://colab.research.google.com/github/Wissg/Trading-code/blob/main/strategyMACD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import



In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
import datetime 
from datetime import date

%config InlineBackend.figure_format = 'retina'

import plotly.graph_objects as go
from plotly.subplots import make_subplots

print('✔️ Libraries Loaded!')

✔️ Libraries Loaded!


#strategy


In [2]:
def get_trading_strategy(df, column='Adj Close'):
    """Return the Buy/Sell signal on the specified (price) column (Default = 'Adj Close')."""
    buy_list, sell_list = [], []
    flag = False

    for i in range(0, len(df)):
        if df['MACD'].iloc[i] > df['Signal'].iloc[i] and flag == False:
            buy_list.append(df[column].iloc[i])
            sell_list.append(np.nan)
            flag = True

        elif df['MACD'].iloc[i] < df['Signal'].iloc[i] and flag == True:
            buy_list.append(np.nan)
            sell_list.append(df[column].iloc[i])
            flag = False

        else:
            buy_list.append(np.nan)
            sell_list.append(np.nan)

    df['Buy'] = buy_list
    df['Sell'] = sell_list

    return df


print('✔️ Function defined!')

✔️ Function defined!


#RSI

In [3]:
def get_RSI(df, column='Adj Close', rsi_period=14):
    """Return the RSI indicator for the specified time window."""
    # Compute the RSI rsi_period
    delta = df[column].diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)

    avg_gain = gain.rolling(window=rsi_period).mean()
    avg_loss = loss.rolling(window=rsi_period).mean()

    rs = abs(avg_gain / avg_loss)
    df['RSI'] = 100 - (100 / (1 + rs))

    return df


print('✔️ Function defined!')

✔️ Function defined!


In [4]:
def plot_RSI(fig, df, row, column=1):
    """Return a graph object figure containing the RSI indicator in the specified row."""
    fig.add_trace(go.Scatter(x=df['Date'].iloc[30:],
                             y=df['RSI'].iloc[30:],
                             name='RSI',
                             line=dict(color='gold', width=2)),
                  row=row,
                  col=column)

    fig.update_yaxes(title_text='RSI', row=row, col=column)

    # Add one red horizontal line at 70% (overvalued) and green line at 30% (undervalued)
    for y_pos, color in zip([70, 30], ['Red', 'Green']):
        fig.add_shape(x0=df['Date'].iloc[1],
                      x1=df['Date'].iloc[-1],
                      y0=y_pos,
                      y1=y_pos,
                      type='line',
                      line=dict(color=color, width=2),
                      row=row,
                      col=column)

    # Add a text box for each line
    for y_pos, text, color in zip([64, 36], ['Overvalued', 'Undervalued'], ['Red', 'Green']):
        fig.add_annotation(x=df['Date'].iloc[int(df['Date'].shape[0] / 10)],
                           y=y_pos,
                           text=text,
                           font=dict(size=14, color=color),
                           bordercolor=color,
                           borderwidth=1,
                           borderpad=2,
                           bgcolor='lightsteelblue',
                           opacity=0.75,
                           showarrow=False,
                           row=row,
                           col=column)

    # Update the y-axis limits
    ymin = 25 if df['RSI'].iloc[30:].min() > 25 else df['RSI'].iloc[30:].min() - 5
    ymax = 75 if df['RSI'].iloc[30:].max() < 75 else df['RSI'].iloc[30:].max() + 5
    fig.update_yaxes(range=[ymin, ymax], row=row, col=column)

    return fig


print('✔️ Function defined!')

✔️ Function defined!


#MACD


In [5]:
def get_MACD(df, column='Adj Close'):
    """Return a new DataFrame with the MACD and related information (signal line and histogram)."""
    df['EMA-12'] = df[column].ewm(span=12, adjust=False).mean()
    df['EMA-26'] = df[column].ewm(span=26, adjust=False).mean()

    # MACD Indicator = 12-Period EMA − 26-Period EMA.
    df['MACD'] = df['EMA-12'] - df['EMA-26']

    # Signal line = 9-day EMA of the MACD line.
    df['Signal'] = df['MACD'].ewm(span=9, adjust=False).mean()

    # Histogram = MACD - Indicator.
    df['Histogram'] = df['MACD'] - df['Signal']

    return df


print('✔️ Function defined!')

✔️ Function defined!


In [6]:
def plot_MACD(fig, df, row, column=1):
    """Return a graph object figure containing the MACD indicator, the signal line, and a histogram in the specified row."""
    df['Hist-Color'] = np.where(df['Histogram'] < 0, 'red', 'green')
    fig.add_trace(go.Bar(x=df['Date'],
                         y=df['Histogram'],
                         name='Histogram',
                         marker_color=df['Hist-Color'],
                         showlegend=True),
                  row=row,
                  col=column)

    fig.add_trace(go.Scatter(x=df['Date'],
                             y=df['MACD'],
                             name='MACD',
                             line=dict(color='darkorange', width=2)),
                  row=row,
                  col=column)

    fig.add_trace(go.Scatter(x=df['Date'],
                             y=df['Signal'],
                             name='Signal',
                             line=dict(color='cyan', width=2)),
                  row=row,
                  col=column)

    fig.update_yaxes(title_text='MACD', row=row, col=column)

    return fig


print('✔️ Function defined!')

✔️ Function defined!


#Closed Date

In [7]:
def get_closed_dates(df):
    """Return a list containing all dates on which the stock market was closed."""
    # Create a dataframe that contains all dates from the start until today.
    timeline = pd.date_range(start=df['Date'].iloc[0], end=df['Date'].iloc[-1])

    # Create a list of the dates existing in the dataframe.
    df_dates = [day.strftime('%Y-%m-%d') for day in pd.to_datetime(df['Date'])]

    # Finally, determine which dates from the 'timeline' do not exist in our dataframe.
    closed_dates = [
        day for day in timeline.strftime('%Y-%m-%d').tolist()
        if not day in df_dates
    ]

    return closed_dates


print('✔️ Function defined!')

✔️ Function defined!


#Price from data

In [8]:
def get_price(ticker, start_date, end_date):
    """Return a DataFrame with price information (open, high, low, close, adjusted close, and volume) for the ticker between the specified dates."""
    df = yf.download(ticker, start_date, end_date, progress=False)
    df.reset_index(inplace=True)

    return df

print('✔️ Function defined!')


✔️ Function defined!


#Volume

In [9]:
def plot_volume(fig, df, row, column=1):
    """Return a graph object figure containing the volume chart in the specified row."""
    fig.add_trace(go.Bar(x=df['Date'],
                         y=df['Volume'],
                         marker=dict(color='lightskyblue',
                                     line=dict(color='firebrick', width=0.1)),
                         showlegend=False,
                         name='Volume'),
                  row=row,
                  col=column)

    fig.update_xaxes(title_text='Date', row=row, col=column)
    fig.update_yaxes(title_text='Volume ($)', row=row, col=column)

    return fig


print('✔️ Function defined!')

✔️ Function defined!


#Main

In [10]:
start_date = datetime.datetime(2022, 1, 1)
ticker = 'AAPL'
no_years = 1

end_date = datetime.datetime.now().strftime('%Y-%m-%d')
start_date = (datetime.datetime.now() -
              datetime.timedelta(days=no_years * 365)).strftime('%Y-%m-%d')


# start_date = datetime.datetime(2022, 1, 1)

print('Ticker: {}'.format(ticker))
print('Start Date: ', start_date)
print('  End Date: ', end_date)

df = get_price(ticker, start_date, end_date)
closed_dates_list = get_closed_dates(df)

print('\n(Raw) Dataset Loaded!')
print('Last five rows:')
df.tail()

Ticker: AAPL
Start Date:  2022-05-03
  End Date:  2023-05-03

(Raw) Dataset Loaded!
Last five rows:


,Date,Open,High,Low,Close,Adj Close,Volume
246,2023-04-26,163.059998,165.279999,162.800003,163.759995,163.759995,45498800
247,2023-04-27,165.190002,168.559998,165.190002,168.410004,168.410004,64902300
248,2023-04-28,168.490005,169.850006,167.880005,169.679993,169.679993,55209200
249,2023-05-01,169.279999,170.449997,168.639999,169.589996,169.589996,52472900
250,2023-05-02,170.089996,170.350006,167.539993,168.539993,168.539993,48329100


In [11]:
df = get_MACD(df)
df = get_RSI(df)

df.tail()

,Date,Open,High,Low,Close,Adj Close,Volume,EMA-12,EMA-26,MACD,Signal,Histogram,RSI
246,2023-04-26,163.059998,165.279999,162.800003,163.759995,163.759995,45498800,164.456604,162.240657,2.215947,2.761981,-0.546034,50.000000
247,2023-04-27,165.190002,168.559998,165.190002,168.410004,168.410004,64902300,165.064820,162.697646,2.367174,2.683020,-0.315846,58.549934
248,2023-04-28,168.490005,169.850006,167.880005,169.679993,169.679993,55209200,165.774846,163.214857,2.559990,2.658414,-0.098424,68.595047
249,2023-05-01,169.279999,170.449997,168.639999,169.589996,169.589996,52472900,166.361792,163.687089,2.674703,2.661672,0.013032,72.619669
250,2023-05-02,170.089996,170.350006,167.539993,168.539993,168.539993,48329100,166.696900,164.046564,2.650337,2.659405,-0.009068,71.334667


In [12]:
df = get_trading_strategy(df)

print('✔️ Final DataFrame is ready!')

✔️ Final DataFrame is ready!


In [13]:
def plot_candlestick_chart(fig, df, row, column=1, plot_EMAs=True, plot_strategy=True):
    """Return a graph object figure containing a Candlestick chart in the specified row."""
    fig.add_trace(go.Candlestick(x=df['Date'],
                                 open=df['Open'],
                                 high=df['High'],
                                 low=df['Low'],
                                 close=df['Close'],
                                 name='Candlestick Chart'),
                  row=row,
                  col=column)

    # If the boolean argument plot_EMAs is True, then show the line plots for the two exponential moving averages.
    if (plot_EMAs == True):
        fig.add_trace(go.Scatter(x=df['Date'],
                                 y=df['EMA-12'],
                                 name='12-period EMA',
                                 line=dict(color='dodgerblue', width=2)),
                      row=row,
                      col=column)

        fig.add_trace(go.Scatter(x=df['Date'],
                                 y=df['EMA-26'],
                                 name='26-period EMA',
                                 line=dict(color='whitesmoke', width=2)),
                      row=row,
                      col=column)

    if (plot_strategy == True):
        fig.add_trace(go.Scatter(x=df['Date'],
                                 y=df['Buy'],
                                 name='Buy Signal',
                                 mode='markers',
                                 marker_symbol='triangle-up',
                                 marker=dict(size=9),
                                 line=dict(color='Lime')),
                      row=row,
                      col=column)

        fig.add_trace(go.Scatter(x=df['Date'],
                                 y=df['Sell'],
                                 name='Sell Signal',
                                 mode='markers',
                                 marker_symbol='triangle-down',
                                 marker=dict(size=9, color='Yellow')),
                      row=row,
                      col=column)

    fig.update_xaxes(rangeslider={'visible': False})
    fig.update_yaxes(title_text='Price ($)', row=row, col=column)

    return fig


print('✔️ Function defined!')

✔️ Function defined!


In [14]:
fig = make_subplots()
fig = plot_candlestick_chart(fig,
                             df,
                             row=1,
                             plot_EMAs=False,
                             plot_strategy=False)
fig.show()

In [15]:
########## Plot the four plots ##########
fig = make_subplots(rows=4,
                    cols=1,
                    shared_xaxes=True,
                    vertical_spacing=0.005,
                    row_width=[0.2, 0.3, 0.3, 0.8])

fig = plot_candlestick_chart(fig,
                             df,
                             row=1,
                             plot_EMAs=True,
                             plot_strategy=True)
fig = plot_MACD(fig, df, row=2)
fig = plot_RSI(fig, df, row=3)
fig = plot_volume(fig, df, row=4)

########## Customise the figure ##########
# Update xaxis properties
fig.update_xaxes(rangebreaks=[dict(values=closed_dates_list)],
                 range=[df['Date'].iloc[0] - datetime.timedelta(days=3), df['Date'].iloc[-1] + datetime.timedelta(days=3)])

# Update basic layout properties (width&height, background color, title, etc.)
fig.update_layout(width=800,
                  height=800,
                  plot_bgcolor='#0E1117',
                  paper_bgcolor='#0E1117',
                  title={
                      'text': '{} - Stock Dashboard'.format(ticker),
                      'y': 0.98
                  },
                  hovermode='x unified',
                  legend=dict(orientation='h',
                              xanchor='left',
                              x=0.05,
                              yanchor='bottom',
                              y=1.003))

# Customize axis parameters
axis_lw, axis_color = 2, 'white'
fig.update_layout(xaxis1=dict(linewidth=axis_lw,
                              linecolor=axis_color,
                              mirror=True,
                              showgrid=False),
                  yaxis1=dict(linewidth=axis_lw,
                              linecolor=axis_color,
                              mirror=True,
                              showgrid=False),
                  font=dict(color=axis_color))

fig.update_layout(xaxis2=dict(linewidth=axis_lw,
                              linecolor=axis_color,
                              mirror=True,
                              showgrid=False),
                  yaxis2=dict(linewidth=axis_lw,
                              linecolor=axis_color,
                              mirror=True,
                              showgrid=False),
                  font=dict(color=axis_color))

fig.update_layout(xaxis3=dict(linewidth=axis_lw,
                              linecolor=axis_color,
                              mirror=True,
                              showgrid=False),
                  yaxis3=dict(linewidth=axis_lw,
                              linecolor=axis_color,
                              mirror=True,
                              showgrid=False),
                  font=dict(color=axis_color))

fig.update_layout(xaxis4=dict(linewidth=axis_lw,
                              linecolor=axis_color,
                              mirror=True,
                              showgrid=False),
                  yaxis4=dict(linewidth=axis_lw,
                              linecolor=axis_color,
                              mirror=True,
                              showgrid=False),
                  font=dict(color=axis_color))

fig.show()